# Gravitational Simulation

**Arnav Maniar — Spring 2024**

**PH 4180: Astrophysics — Happer**

## Introduction

This project is a gravitational simulation built using Pygame. It visualizes the interactions between the earth, moon, and spacecraft, under the influence of gravity. Users can create spacecraft and observe their trajectories as they orbit around planets or move through space. The simulation uses fundamental physics principles, including Newton's law of universal gravitation and centripetal acceleration, to accurately model the behavior of objects in space. Through interactive controls and real-time visual feedback, users can explore the dynamics of gravitational forces and visualize orbital mechanics.

## Import Section

In [1]:
import pygame
import math
import pygame_gui

pygame-ce 2.4.1 (SDL 2.28.5, Python 3.9.6)


#### Initializing pygame

In [2]:
pygame.init()

(5, 0)

#### Initializing constants for pygame

In [3]:
width, height = 1400, 800
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("Gravitational Simulation")

2024-05-13 15:27:38.445 Python[26794:11146762] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


: 

#### Initializing Constants for Simulation

In [ ]:
plan_mass = 200  #arbitrary value (Earth does not have a mass of 200kg)
ship_mass = 1
G = 10
FPS = 60
planet_size = 50
object_size = 5
velocity_scale = 50

#### Load Supplementary Images & Initialize Manager

In [ ]:
bg = pygame.transform.scale(pygame.image.load("bkg.jpg"), (width, height))
planet = pygame.transform.scale(pygame.image.load("earth.png"), (planet_size * 2, planet_size * 2))
moon = pygame.transform.scale(pygame.image.load("moon.jpeg"), (planet_size, planet_size))
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
manager = pygame_gui.UIManager((width, height))

## Initializing Classes

#### Planet Class

This class below initializes a planet object with the given position and mass. 

- x: The x-coordinate of the planet's center. 
- y: The y-coordinate of the planet's center. 
- mass: The mass of the planet. 

It creates instances of the Planet class with specific attributes: x and y coordinates representing the position of the planet's center, and mass representing the mass of the planet. These attributes are assigned to the object using **self** to make them accessible within the class.

The draw method renders the planet on the Pygame window. It uses the **blit** function  of the Pygame library to draw the planet image on the window surface. The planet is drawn at coordinates  `(self.x - planet_size, self.y - planet_size)`, which ensures that the center of the planet aligns with its specified position `(x, y)` on the window. The `planet_size` variable is subtracted to adjust for the size of the planet image.

In [ ]:
class Planet:
    def __init__(self, x, y, mass):
        self.x = x
        self.y = y
        self.mass = mass
    def draw(self):
        win.blit(planet, (self.x - planet_size, self.y - planet_size))

#### Moon Class

The code below is just to intialize each of the variables using self, as we did in the Planet class.

In [ ]:
class Moon:
    def __init__(self, x, y, mass, distance_from_planet, angular_velocity):
        self.x = x
        self.y = y
        self.mass = mass
        self.distance_from_planet = distance_from_planet
        self.angular_velocity = angular_velocity
        self.angle = 0  #initial angle

#### Updating Moon's Position:
In the `update_position` method of the Moon class, the moon's position is updated based on its angular velocity and distance from the planet. The key mathematical operations involved are:

- Calculating New Angle: The angle of the moon (`self.angle`) is incremented by its angular velocity (`self.angular_velocity`). This represents the moon's angular displacement over a certain time interval.

- Using Trigonometric Functions: The new x and y coordinates of the moon are calculated using trigonometric functions `math.cos()` and `math.sin()`, respectively. These functions relate the angle (in radians) to the coordinates on a circle (or orbit).

- Coordinate Calculation: The updated x coordinate of the moon is calculated as follows:
    `self.x = planet.x + self.distance_from_planet * math.cos(self.angle)`
- Similarly, the updated y coordinate is calculated as:
    `self.y = planet.y + self.distance_from_planet * math.sin(self.angle)`

In [ ]:
    def update_position(self, planet):
        #update angle based on angular velo
            self.angle += self.angular_velocity
        
        #calc new pos based on angle/distance from planet
            self.x = planet.x + self.distance_from_planet * math.cos(self.angle)
            self.y = planet.y + self.distance_from_planet * math.sin(self.angle)

#### Drawing Moon
This method renders the moon on the Pygame window by blitting its image onto the window surface.  The moon is drawn at coordinates `(self.x - planet_size // 2, self.y - planet_size // 2)`, so that its center aligns with its position on the window. The `planet_size` variable is divided by 2 to adjust for the size of the moon image. 

In [ ]:
def draw(self):
        win.blit(moon, (int(self.x) - planet_size // 2, int(self.y) - planet_size // 2))

#### Spacecraft Class

The code below initializes a variable called `spacecraft_count` that keeps track of the active number of spacecraft. The self word just assigns them to the object.

In [ ]:
class Spacecraft:
    spacecraft_count = 0
    def __init__(self, x, y, vel_x, vel_y, mass):
        self.x = x 
        self.y = y
        self.vel_x = vel_x
        self.vel_y = vel_y
        self.mass = mass
        self.in_orbit = False  # orbit flag
        Spacecraft.spacecraft_count += 1 #add 1 to counter 

#### Calculating Gravitational Force and Acceleration for Spacecraft:

In the move method of the Spacecraft class, the gravitational force and acceleration from the planet are calculated to update the spacecraft's velocity. The relevant mathematical concepts include:

**Newton's Law of Universal Gravitation:** The gravitational force between two objects with masses m1 and m2 separated by a distance r is given by:

$$
F = \frac{G \cdot m_1 \cdot m_2}{r^2}
$$


**Where:**

- 𝐹 is the gravitational force.
- 𝐺 is the gravitational constant.
- $𝑚_{1}$ and $𝑚_{2}$ are the masses of the objects.
- 𝑟 is the distance between the objects.

Newton's Second Law: The force experienced by an object is equal to its mass multiplied by its acceleration $F = m \cdot a$. Rearranging this equation, we get: 
$$
a = \frac{F}{m}
$$

**Components of Acceleration:** Since acceleration is a vector quantity, it has both horizontal and vertical components. These components are calculated using trigonometric functions based on the angle between the spacecraft and the planet.

**Updating Velocity:** The horizontal and vertical components of acceleration are added to the spacecraft's current velocities (`self.vel_x` and `self.vel_y`, respectively) to update its velocity in each frame.


#### Calculating Orbital Velocity and Acceleration for Spacecraft:

**Orbital Velocity Calculation:** When the spacecraft enters orbit around the planet, its velocity is adjusted to achieve a stable orbit. This velocity is calculated using the formula:
$$
v_{\text{orbit}} = \frac{r}{\sqrt{G \cdot M}}
$$

**Where:**

- $𝑣_{orbit}$ is the orbital velocity.
- 𝐺 is the gravitational constant.
- 𝑀 is the mass of the planet.
- 𝑟 is the distance from the spacecraft to the planet's center.

**Centripetal Acceleration Calculation:** In circular motion, centripetal acceleration $a_{c}$ is the acceleration directed towards the center of the circular path. It is calculated using the formula:

$$
a_c = \frac{v^2}{r}
$$

**Where:**

- 𝑣 is the velocity of the spacecraft.
- 𝑟 is the distance from the spacecraft to the center of the orbit.

**Updating Velocity and Position:** The spacecraft's velocity is adjusted to the orbital velocity, and its position is updated accordingly to enter a stable orbit around the planet.

The `math.atan2()` function calculates the angle between two points in the Cartesian coordinate system using the arctangent function. It takes two arguments: the difference in y-coordinates ($\Delta y = \text{planet.y} - \text{self.y}$) and the difference in x-coordinates ($\Delta x = \text{planet.x} - \text{self.x}$). The function returns the angle $\theta$ between the positive x-axis and the line connecting the two points, measured counterclockwise from the positive x-axis. Mathematically, $\theta = \text{atan2}(\Delta y, \Delta x)$. The calculated angle represents the direction of the gravitational force from the planet acting on the spacecraft. Using trigonometric functions, the horizontal ($a_x$) and vertical ($a_y$) components of acceleration are computed as $a_x = a \cdot \cos(\theta)$ and $a_y = a \cdot \sin(\theta)$, respectively, where $a$ is the total acceleration from the planet.



In [ ]:
def move(self, planet, moon=None):
        distance_to_planet = math.sqrt((self.x - planet.x)**2 + (self.y - planet.y)**2)
        #if spacecraft is not in orbit but is within 100px:
        if not self.in_orbit and distance_to_planet <= 100:
            angle_to_planet = math.atan2(planet.y - self.y, planet.x - self.x) #finds angle based on positions
            self.in_orbit = True  #orbit flag true
            orbit_velocity = math.sqrt(G * planet.mass / distance_to_planet)
            self.vel_x = orbit_velocity * math.cos(angle_to_planet + math.pi / 2) 
            self.vel_y = orbit_velocity * math.sin(angle_to_planet + math.pi / 2) 
        else:
        #if not in 100px, calculate grav force
            force_from_planet = (G * self.mass * planet.mass) / distance_to_planet ** 2 #universal law of gravitation
            acceleration_from_planet = force_from_planet / self.mass # f= ma
            angle_to_planet = math.atan2(planet.y - self.y, planet.x - self.x) #angle using tangent

            acc_x_from_planet = acceleration_from_planet * math.cos(angle_to_planet)
            acc_y_from_planet = acceleration_from_planet * math.sin(angle_to_planet) #how much zoom does this have

            self.vel_x += acc_x_from_planet #zoom + zoom zoom
            self.vel_y += acc_y_from_planet #zoom + zoom zoom but other way

#### Various Functions within Spacecraft Class

- `get_velocity(self)`:

    * This method calculates and returns the magnitude of the velocity vector of the spacecraft.
    * It uses the Pythagorean theorem to compute the magnitude of the velocity vector from its horizontal and vertical components.
    * The formula used is $vel_x^2 + vel_y^2$, where $vel_x$ and $vel_y$ are the horizontal and vertical components of velocity, respectively.
    * This method effectively determines the overall speed of the spacecraft regardless of its direction.
- `get_centripetal_acceleration(self, planet)`:
    * This method calculates the centripetal acceleration experienced by the spacecraft as it orbits around the planet.
    * Then, it uses the formula $a_c = \frac{v^2}{r}$ to calculate the centripetal acceleration, where  𝑣 is the velocity magnitude obtained from the `get_velocity()` method.
-`get_force_from_planet(self, planet)`:
    * This method calculates the gravitational force exerted on the spacecraft by the planet using Newton's Law of Universal Gravitation. It first computes the distance 𝑟 between the spacecraft and the planet using the formula for Euclidean distance.
    * Then, it applies the formula $G \cdot m_s \cdot m_p / r^2$ to determine the gravitational force, where $G$ is the gravitational constant, $m_s$ is the mass of the spacecraft, $m_p$ is the mass of the planet, and $r$ is the distance between them. The function then returns the calculated gravitational force.
- `draw(self)`:
    * This method is responsible for rendering the spacecraft on the Pygame window.
    * It draws a circle representing the spacecraft at its current position on the window.
    * The color of the circle is defined as red, and the position is determined by the spacecraft's coordinates (`x` and `y`) adjusted to integers.
    * The circle's size is determined by the `object_size` constant defined earlier in the code.
    * This method visually represents the spacecraft within the simulation environment.

In [1]:
    def get_velocity(self):
        return math.sqrt(self.vel_x ** 2 + self.vel_y ** 2) #uses pythag to return velocity

    def get_centripetal_acceleration(self, planet):
        distance_to_planet = math.sqrt((self.x - planet.x)**2 + (self.y - planet.y)**2) #r
        return self.get_velocity()**2 / distance_to_planet #v^2 / r

    def get_force_from_planet(self, planet):
        distance_to_planet = math.sqrt((self.x - planet.x)**2 + (self.y - planet.y)**2)
        return (G * self.mass * planet.mass) / distance_to_planet ** 2 #universal law of grav

    def draw(self):
        pygame.draw.circle(win, RED, (int(self.x), int(self.y)), object_size) #draws spacecraft (its a dot but ykwhat i mean)

## Creating Spacecraft

The `location` parameter contains the coordinates of the starting position (`tx, ty`) of the spacecraft, while the mouse parameter contains the coordinates of the `mouse` position (`mx, my`) where the spacecraft's velocity is directed.
The velocity of the spacecraft is determined by the difference between the mouse coordinates and the starting position coordinates, divided by the `velocity_scale` constant. The velocity scale constant is defined in the initializing section and can be tweaked. Essentially it tells the program how much velocity to scale by per increment of the length the mouse is dragged.
The difference in x-coordinates `(mx - tx)` represents the horizontal displacement between the mouse position and the starting position.
The difference in y-coordinates `(my - ty)` represents the vertical displacement.
Dividing these displacements by `velocity_scale` scales down the velocity to ensure it remains within manageable limits. It normalizes the velocity vector and the scaling allows for better control over the velocity of the spacecraft.
Using the calculated velocity components (`vel_x` and `vel_y`), along with the starting position (`tx` and `ty`), a new instance of the `Spacecraft` class is created.
The `Spacecraft` object is initialized with these parameters, including the mass of the spacecraft (`ship_mass`), and is returned from the function.

In [2]:
def create_ship(location, mouse):
    tx, ty = location 
    mx, my = mouse 
    vel_x = (mx - tx) / velocity_scale
    vel_y = (my - ty) / velocity_scale
    obj = Spacecraft(tx, ty, vel_x, vel_y, ship_mass) #this is so cool i used the class finally omg
    return obj

#### Clear Objects

This function will later be used so that when a button is clicked, this function is called. It removes all spacecraft from the screen and also sets the counter to 0.

In [ ]:
def clear_objects(objects):
    objects.clear() #clear
    Spacecraft.spacecraft_count = 0


## Main Function


#### Initialization
This code primarily serves to initialize objects and use the classes. Various variables are defined and the planet and moon objects are created. The Planet object takes in parameters for its location and is put at the center of the user's screen. This is almost the same for the moon, however it is placed 200 pixels to the right, and it rotates around the earth with a ${r}$ of 200. The clear button is also created here.

In [4]:
def main():
    running = True
    clock = pygame.time.Clock()

    planet_obj = Planet(width // 2, height // 2, plan_mass)
    moon_obj = Moon(width // 2 + 200, height // 2, 25, 200, 0.01) #USING THE CLASS LETS GOOOOO
    objects = []
    temp_pos = None
    new_object = None
    font = pygame.font.Font(None, 36) #font1 for spacecraft counter
    font2 = pygame.font.Font(None,22) #font2 for data on orbit text
    #make button to clear gui
    button_clear = pygame_gui.elements.UIButton(relative_rect=pygame.Rect((width-150, 50), (100, 50)),
                                                 text='Clear', manager=manager)

#### While Loop

This loop is primarily for event handling, however it also keeps the framerate of the game running smoothly. Pygame checks for any button presses or mouse clicks, and it calls the appropriate function as a result. For instance, if the clear button is pressed, it calls the `clear_objects` function, which removes all objects. If the left mouse button is pressed, it records the current mouse position (`event.pos`) as the temporary position (`temp_pos`). If the right mouse button is pressed and a temporary position is recorded, it calls the `create_ship` function to create a new spacecraft object (`new_object`) with the temporary position and the current mouse position. If the right mouse button is released, it appends the newly created spacecraft object (`new_object`) to the list of objects (`objects`).

Lastly, the program blits the background to the window and renders it. This is also where the spacecraft counter in the bottom left is created and updated. 

In [ ]:
    while running:
        time_delta = clock.tick(FPS) / 1000.0
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.USEREVENT:
                if event.user_type == pygame_gui.UI_BUTTON_PRESSED:
                    if event.ui_element == button_clear:
                        clear_objects(objects) #clears
            if event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    temp_pos = event.pos
                elif event.button == 3:
                    if temp_pos:
                        new_object = create_ship(temp_pos, event.pos) #makes spacecrafat
            if event.type == pygame.MOUSEBUTTONUP:
                if event.button == 3:
                    if new_object:
                        objects.append(new_object)
                        #new_object = None
                        #initial_pos = None
            manager.process_events(event)
    win.blit(bg, (0, 0))

    spacecraft_count_text = font.render(f"Spacecraft Active: {Spacecraft.spacecraft_count}", True, WHITE)
    win.blit(spacecraft_count_text, (10, height - 40)) #blits the counter in bototm left


#### Temporary Position

This code is still part of the while loop. 

The if `temp_pos`: condition checks if a temporary position is recorded (if the left mouse button is pressed).
If `temp_pos` exists, a line is drawn from the temporary position (`temp_pos`) to the current mouse position (`pygame.mouse.get_pos()`). This line represents the path the spacecraft would take if launched from the temporary position.
Additionally, a blue circle is drawn at the temporary position (`temp_pos`) to visualize the starting point of the spacecraft.

**Updating and Drawing Spacecraft:**
After that, to update the positions, the loop iterates over each spacecraft object (`obj`) in the `objects` list.
For each spacecraft, its `draw()` method is called to render it on the screen.
The `move()` method of the spacecraft is called to update its position based on the gravitational forces from the planet and moon.

**Collision Detection**
The code checks for three conditions to determine if a spacecraft has collided or gone off-screen:
- `off_screen`: Checks if the spacecraft's position is outside the window boundaries.
- `collided_with_planet`: Calculates the distance between the spacecraft and the planet's center and checks if it's less than or equal to the planet's radius (`planet_size`). If true, it indicates a collision with the planet.
- `collided_with_moon`: Similarly, calculates the distance between the spacecraft and the moon's center and checks for collision.
If any of these conditions are met, it removes the spacecraft and subtracts 1 from the spacecraft counter.

In [ ]:
if temp_pos:
            #draws the user figuring out how long and far they want to send the spacecraft
            pygame.draw.line(win, WHITE, temp_pos, pygame.mouse.get_pos(), 2) 
            pygame.draw.circle(win, BLUE, temp_pos, object_size)
        
        for obj in objects[:]:
            obj.draw()
            obj.move(planet_obj, moon_obj)
            off_screen = obj.x < 0 or obj.x > width or obj.y < 0 or obj.y > height 
            collided_with_planet = math.sqrt((obj.x - planet_obj.x)**2 + (obj.y - planet_obj.y)**2) <= planet_size #checks for collision
            collided_with_moon = math.sqrt((obj.x - moon_obj.x)**2 + (obj.y - moon_obj.y)**2) <= planet_size 
            if off_screen or collided_with_planet or collided_with_moon:
                objects.remove(obj)
                Spacecraft.spacecraft_count -= 1 #removes 1 from spacecraft counter

#### Draw New Planets

After updates, the planets are redrawn.

In [ ]:
        planet_obj.draw() #draw
        moon_obj.draw() #DRAW
        moon_obj.update_position(planet_obj) #UPDATE IT

#### Displaying Information for Spacecraft in Orbit

**Condition Check**
`if new_object is not None and new_object.in_orbit:`: This condition checks if `new_object` exists and if it is in orbit around the planet. It makes sure that the following calculations and display are performed only when a new spacecraft is created and successfully enters orbit.
**Calculations**
- `velocity = new_object.get_velocity()`: Calculates the velocity of the spacecraft using the `get_velocity()` method of the spacecraft object.
- `centripetal_acceleration = new_object.get_centripetal_acceleration(planet_obj)`: Calculates the centripetal acceleration experienced by the spacecraft using the `get_centripetal_acceleration()` method of the spacecraft object.
- `force_from_planet = new_object.get_force_from_planet(planet_obj)`: Calculates the gravitational force exerted on the spacecraft by the planet using the `get_force_from_planet()` method of the spacecraft object.
**Text Rendering**
`velocity_text`, `acceleration_text`, and `force_text` are created using the `font2.render()` method. This renders the calculated velocity, centripetal acceleration, and gravitational force, in the top right, and all are rounded to 2 decimal places. They are blited onto the window and positioned 15 pixels away from each other in the y.

In [ ]:
if new_object is not None and new_object.in_orbit:
            velocity = new_object.get_velocity() #returns v
            centripetal_acceleration = new_object.get_centripetal_acceleration(planet_obj) #returns ac
            force_from_planet = new_object.get_force_from_planet(planet_obj) #returns f
            
            velocity_text = font2.render(f"Velocity: {velocity:.2f} pixels/frame", True, WHITE) #defines render
            acceleration_text = font2.render(f"Centripetal Acceleration: {centripetal_acceleration:.2f} pixels/frame^2", True, WHITE)
            force_text = font2.render(f"Force from Planet: {force_from_planet:.2f} Newtons (scaled)", True, WHITE)
            
            win.blit(velocity_text, (10, 10)) #blits render
            win.blit(acceleration_text, (10, 25))
            win.blit(force_text, (10, 40))


        manager.update(time_delta)
        manager.draw_ui(win)

        pygame.display.update()

    pygame.quit()


## Ending + Calling

This final portion of the code simply calls the main function and runs the code. We're done!

In [ ]:

if __name__ == "__main__":
    main()